In [1]:
import pandas as pandas
import nltk
from nltk.corpus import stopwords
import time
import re
import numpy as np
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.stem.porter import PorterStemmer 

import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')

In [2]:
import gensim

In [3]:
filename = 'train_data.csv'
t1 = time.time()
data = pandas.read_csv('train_data.csv')

In [4]:
data_5=data[data["stars"]==5]
data_txt5=data_5["text"]
data_txt5=data_txt5[0:50]
data_txt5.to_csv("5 star.csv",sep=",")

data_1=data[data["stars"]==1]
data_txt1=data_1["text"]
data_txt1=data_txt1[3:53]
data_txt1.to_csv("1 star.csv",sep=",")

In [5]:
#nltk.download()

In [6]:
stop = set(stopwords.words("english"))

data_modeltxt=data["text"]
#data_modeltxt=data_modeltxt[0:1000]
data_modeltxt.head()
    
def stop_delete(sentence):
    sentence=sentence.lower().split()
    sen=[word for word in sentence if not word in stop]
    sen_re=np.array([])
    for j in sen:
        word_re=re.sub('[^a-zA-Z]','', j)
        if word_re != "":
            sen_re=np.append(sen_re,word_re)
    return sen_re

In [ ]:
from gensim import corpora
from collections import defaultdict

lem = WordNetLemmatizer()
stem = PorterStemmer()

document=[];
for i in data_modeltxt:
    txt=stop_delete(i)
    txt=[stem.stem(word) for word in txt]
    document.append(txt);

# remove words that appear only once
frequency=defaultdict(int)
for txt in document:
    for word in txt:
        frequency[word] += 1

document = [[word for word in txt if frequency[word] > 1] for txt in document]

In [ ]:
#dictionary
dictionary = corpora.Dictionary(document)
dictionary.save('yelpword.dict')
#list to corpus
corpus = [dictionary.doc2bow(txt) for txt in document]
corpora.MmCorpus.serialize('yelpreview.mm', corpus)
document = corpora.MmCorpus('yelpreview.mm')

In [ ]:
LDA = gensim.models.ldamodel.LdaModel(corpus=document, id2word=dictionary, num_topics=100)

In [ ]:
LDA.print_topics(5)